In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text=open(path_to_file,'rb').read().decode(encoding='utf-8')
print(len(text))

1115394


In [4]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [5]:
vocab=sorted(set(text))

char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[x] for x in text])

text_as_int=text_to_int(text)  

In [6]:
print(text[:13])
print(text_to_int(text[:13]))

First Citizen
[18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints=ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [8]:
seq_length=100
examples_per_epoch=len(text)//(seq_length+1)

char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)
sequences=char_dataset.batch(seq_length+1,drop_remainder=True)

In [9]:
def split_input_target(chunk):
  input_text=chunk[:-1]
  output_text=chunk[1:]
  return input_text,output_text

dataset=sequences.map(split_input_target)  

In [10]:
batch_size=64
vocab_size=len(vocab)
embedding_dim=256
rnn_units=1024
buffer_size=10000

data=dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [11]:
def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
  model=tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,embedding_dim,
                                  batch_input_shape=[batch_size,None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)                                                  
  ])
  return model

model=build_model(vocab_size,embedding_dim,rnn_units,batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


Creating a Loss Function

In [12]:
for input_example_batch,target_example_batch in data.take(1):
  example_batch_prediction = model(input_example_batch)
  print(example_batch_prediction.shape)

pred=example_batch_prediction[0]
time_pred=pred[0]

(64, 100, 65)


In [13]:
sampled_indices=tf.random.categorical(pred,num_samples=1)

sampled_indices=np.reshape(sampled_indices,(1,-1))[0]
predicted_chars=int_to_text(sampled_indices)

predicted_chars

"-ylL !ASrzHCJKFictcfjMM-FeNDsTq!N:dBrT ,EgLPk&Hnm.Vu.zZ,cru&;Vh$YCEyOpV:n?JGnj3Vk.jAyRZbPIC;.wy\nJ'ag"

In [14]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [15]:
model.compile(optimizer='adam',loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [17]:
history=model.fit(data,epochs=40)

Epoch 1/40
172/172 [==============================] - 33s 171ms/step - loss: 3.0377
Epoch 2/40
172/172 [==============================] - 31s 170ms/step - loss: 1.9630
Epoch 3/40
172/172 [==============================] - 31s 170ms/step - loss: 1.6646
Epoch 4/40
172/172 [==============================] - 31s 171ms/step - loss: 1.5139
Epoch 5/40
172/172 [==============================] - 31s 171ms/step - loss: 1.4269
Epoch 6/40
172/172 [==============================] - 31s 171ms/step - loss: 1.3650
Epoch 7/40
172/172 [==============================] - 31s 172ms/step - loss: 1.3190
Epoch 8/40
172/172 [==============================] - 31s 172ms/step - loss: 1.2807
Epoch 9/40
172/172 [==============================] - 31s 172ms/step - loss: 1.2387
Epoch 10/40
172/172 [==============================] - 31s 171ms/step - loss: 1.2049
Epoch 11/40
172/172 [==============================] - 31s 171ms/step - loss: 1.1712
Epoch 12/40
172/172 [==============================] - 31s 171ms/step - lo

In [18]:
model = build_model(vocab_size,embedding_dim,rnn_units, batch_size=1)

In [27]:
model.build(tf.TensorShape([1, None]))

In [28]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))